referensi https://jundongl.github.io/scikit-feature/tutorial.html

fungsi fisher_score() cuman nerima variabel jenis numpy

note
https://stackoverflow.com/questions/63538875/module-skfeature-function-similarity-based-fisher-score-has-no-attribute-feat

In [1]:
# install package skfeature-chappers
%pip install skfeature-chappers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/tljh/user/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
#libraries buat seleksi fitur
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from skfeature.function.similarity_based import fisher_score

#libraries buat klasifikasi
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import time
import sys

In [10]:
#import dataset
df = pd.read_csv('nirsMangga.csv')

# separate dataset into train and test
x_train, x_test, y_train, y_test = train_test_split(
    df.drop(labels=['No', 'Mango Cultivars', 'Vit C (mg/100g)', 'TA (mg/100g)', 'SSC (oBrix)', 'label'], axis=1),
    df['label'],
    test_size=0.3,
    random_state=0)

x_train.head()

,999.9,1000.3,1000.7,1001.1,1001.4,1001.8,1002.2,1002.6,1003,1003.4,...,2478.7,2481.1,2483.5,2485.8,2488.2,2490.6,2493,2495.4,2497.8,2500.2
16,0.471459,0.471074,0.470934,0.470379,0.470260,0.469880,0.469497,0.469435,0.469454,0.468998,...,1.413537,1.415740,1.417568,1.419698,1.421711,1.423070,1.424394,1.426121,1.427552,1.428625
51,0.433239,0.432622,0.432626,0.432379,0.431620,0.430710,0.430836,0.430847,0.430188,0.429470,...,1.601232,1.602877,1.604524,1.605982,1.606778,1.607837,1.608756,1.609967,1.610900,1.611099
183,0.545045,0.544204,0.543792,0.543596,0.543338,0.542534,0.541493,0.541139,0.541308,0.540831,...,1.524657,1.525973,1.527454,1.529518,1.530097,1.530315,1.530254,1.531191,1.532366,1.533183
145,0.545846,0.544815,0.544524,0.544631,0.544169,0.543143,0.542535,0.542080,0.541842,0.541258,...,1.421962,1.422955,1.423717,1.424639,1.425080,1.425797,1.426503,1.427164,1.427838,1.428271
40,0.381048,0.380483,0.380541,0.380151,0.379599,0.379189,0.379009,0.378722,0.378309,0.377719,...,1.571125,1.572674,1.574303,1.576075,1.577273,1.577980,1.578561,1.579334,1.580042,1.581424


# Seleksi Fitur Fisher Score

In [11]:
%%time

#cari ranking setiap fitur berdasarkan fisher score
fisher_score = fisher_score.fisher_score(x_train.to_numpy(), y_train.to_numpy())

fisher_score

CPU times: user 3.11 s, sys: 4.34 s, total: 7.46 s
Wall time: 581 ms


array([1336, 1335, 1327, ...,    2,    1,    0])

In [14]:
#ini ascending
#bikin dataframe buat nyimpan fitur beserta rankingnya
fisher_ranking = pd.DataFrame(fisher_score)
features = pd.DataFrame(x_train.columns)
fisher_ranking = pd.concat([features,fisher_ranking],axis=1)

#reset index, supaya tidak ada nilai nan di dataframe yg dibikin
fisher_ranking.reset_index(drop=True, inplace=True)
features.reset_index(drop=True, inplace=True)

# Assign the column name
fisher_ranking.columns = ["Input_Features","Fisher_Ranking"]

# Print features score
print(fisher_ranking.nsmallest(100,columns="Fisher_Ranking"))

     Input_Features  Fisher_Ranking
1556         2500.2               0
1555         2497.8               1
1554         2495.4               2
1553           2493               3
1552         2490.6               4
...             ...             ...
1463         2294.5              95
1442         2252.6              96
1431         2231.3              97
1428         2225.5              98
1426         2221.7              99

[100 rows x 2 columns]


In [8]:
print(fisher_ranking.nsmallest(100,columns="Fisher_Ranking").iloc[:,0].values.flatten().tolist())

['2500.2', '2497.8', '2495.4', '2493', '2490.6', '2488.2', '2485.8', '2483.5', '2481.1', '2478.7', '2476.3', '2474', '2471.6', '2469.3', '2466.9', '2464.6', '2462.2', '2459.9', '2457.6', '2455.2', '2452.9', '2448.3', '2427.7', '2414.1', '2405.1', '2398.5', '2389.6', '2380.8', '2372.1', '2365.6', '2357', '2348.5', '2340', '2331.6', '2321.2', '2314.9', '2308.8', '2302.6', '2300.6', '2288.4', '2280.3', '2284.3', '2278.3', '2266.4', '2268.4', '2274.3', '2270.3', '2264.4', '2272.3', '2260.4', '2256.5', '2258.5', '2250.6', '2240.9', '2233.2', '2237', '2235.1', '2239', '2242.8', '2244.8', '2246.7', '2248.7', '2262.4', '2306.7', '2342.1', '2361.3', '2359.2', '2346.4', '2335.8', '2333.7', '2323.2', '2319.1', '2312.9', '2310.8', '2304.7', '2296.5', '2292.4', '2286.4', '2282.3', '2298.5', '2327.4', '2352.8', '2387.4', '2439.1', '2450.6', '2436.8', '2418.6', '2409.6', '2402.9', '2394', '2385.2', '2374.3', '2363.5', '2344.2', '2325.3', '2294.5', '2252.6', '2231.3', '2225.5', '2221.7']


In [11]:
dir(fisher_score)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__

# Klasifikasi Random Forest Classifier

In [20]:
#buat n-fold cv
#cross validation 10-fold
cv = KFold(n_splits=10, random_state=1, shuffle=True)

## Testing Performa Model Random Forest Classifier

In [27]:
%%time

#tentukan metode scoring yang digunakan
scoring_rfe = {'acc': 'accuracy',
               'prec_micro': 'precision_micro',
               'rec_micro': 'recall_micro'}

#tentukan total fitur dan trees yang digunakan dalam proses klasifikasi ini
n_feat = range(1, 101)
n_trees = [100]

max_acc = 0
max_prec = 0
max_rec = 0

for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        start_time = time.time()
        
        #ambil n fitur input hasil seleksi fitur MI
        x_train_selected = x_train[fisher_ranking.nlargest(100,columns="Fisher_Ranking").iloc[0:nfeat, 0]]
        x_test_selected = x_test[fisher_ranking.nlargest(100,columns="Fisher_Ranking").iloc[0:nfeat, 0]]

        #Create a Gaussian Classifier
        clf_rfe = RandomForestClassifier(n_estimators=ntrees)
        
        #Train the model using the training sets
        clf_rfe.fit(x_train_selected, y_train)
        y_pred_rfe=clf_rfe.predict(x_test_selected)
        
        #hitung score model dari data train
        scores_rfe = cross_validate(clf_rfe, x_train_selected, y_train, scoring=scoring_rfe, cv=cv, return_train_score=True)
        
        print("Akurasi model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_acc'].mean(), 2)))
        print("Akurasi model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.accuracy_score(y_test, y_pred_rfe), 2)))
        print("Precision model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_prec_micro'].mean(), 2)))
        print("Precision model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2)))
        print("Recall model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_rec_micro'].mean(), 2)))
        print("Recall model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2)))
        print(" ")
        
        if(round(metrics.accuracy_score(y_test, y_pred_rfe), 2) > max_acc):
            max_acc = round(metrics.accuracy_score(y_test, y_pred_rfe), 2)
        
        if(round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2) > max_prec):
            max_prec = round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2)
            
        if(round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2) > max_rec):
            max_rec = round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2)
            
        end_time = time.time()
        print("Total waktu: ", end_time - start_time)

print("====Nilai Performa Tertinggi====")
print("Nilai akurasi model tertinggi: " + str(max_acc))
print("Nilai presisi model tertinggi: " + str(max_prec))
print("Nilai Recall model tertinggi: " + str(max_rec))

Akurasi model RFE data Train dengan 1 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 1 fitur dan 100 trees:0.84
Precision model RFE data Train dengan 1 fitur dan 100 trees:1.0
Precision model RFE data Test dengan 1 fitur dan 100 trees:0.84
Recall model RFE data Train dengan 1 fitur dan 100 trees:1.0
Recall model RFE data Test dengan 1 fitur dan 100 trees:0.84
 
Total waktu:  1.2842700481414795
Akurasi model RFE data Train dengan 2 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 2 fitur dan 100 trees:0.82
Precision model RFE data Train dengan 2 fitur dan 100 trees:1.0
Precision model RFE data Test dengan 2 fitur dan 100 trees:0.82
Recall model RFE data Train dengan 2 fitur dan 100 trees:1.0
Recall model RFE data Test dengan 2 fitur dan 100 trees:0.82
 
Total waktu:  1.2739362716674805
Akurasi model RFE data Train dengan 3 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 3 fitur dan 100 trees:0.84
Precision model RFE data Train dengan 3 fitur dan 100 tree

Akurasi model RFE data Train dengan 19 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 19 fitur dan 100 trees:0.82
Precision model RFE data Train dengan 19 fitur dan 100 trees:1.0
Precision model RFE data Test dengan 19 fitur dan 100 trees:0.82
Recall model RFE data Train dengan 19 fitur dan 100 trees:1.0
Recall model RFE data Test dengan 19 fitur dan 100 trees:0.82
 
Total waktu:  1.3604085445404053
Akurasi model RFE data Train dengan 20 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 20 fitur dan 100 trees:0.84
Precision model RFE data Train dengan 20 fitur dan 100 trees:1.0
Precision model RFE data Test dengan 20 fitur dan 100 trees:0.84
Recall model RFE data Train dengan 20 fitur dan 100 trees:1.0
Recall model RFE data Test dengan 20 fitur dan 100 trees:0.84
 
Total waktu:  1.344874382019043
Akurasi model RFE data Train dengan 21 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 21 fitur dan 100 trees:0.84
Precision model RFE data Train dengan 21 fitu

Akurasi model RFE data Train dengan 37 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 37 fitur dan 100 trees:0.89
Precision model RFE data Train dengan 37 fitur dan 100 trees:1.0
Precision model RFE data Test dengan 37 fitur dan 100 trees:0.89
Recall model RFE data Train dengan 37 fitur dan 100 trees:1.0
Recall model RFE data Test dengan 37 fitur dan 100 trees:0.89
 
Total waktu:  1.4198663234710693
Akurasi model RFE data Train dengan 38 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 38 fitur dan 100 trees:0.91
Precision model RFE data Train dengan 38 fitur dan 100 trees:1.0
Precision model RFE data Test dengan 38 fitur dan 100 trees:0.91
Recall model RFE data Train dengan 38 fitur dan 100 trees:1.0
Recall model RFE data Test dengan 38 fitur dan 100 trees:0.91
 
Total waktu:  1.3930516242980957
Akurasi model RFE data Train dengan 39 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 39 fitur dan 100 trees:0.91
Precision model RFE data Train dengan 39 fit

Akurasi model RFE data Train dengan 55 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 55 fitur dan 100 trees:0.93
Precision model RFE data Train dengan 55 fitur dan 100 trees:1.0
Precision model RFE data Test dengan 55 fitur dan 100 trees:0.93
Recall model RFE data Train dengan 55 fitur dan 100 trees:1.0
Recall model RFE data Test dengan 55 fitur dan 100 trees:0.93
 
Total waktu:  1.4471139907836914
Akurasi model RFE data Train dengan 56 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 56 fitur dan 100 trees:0.91
Precision model RFE data Train dengan 56 fitur dan 100 trees:1.0
Precision model RFE data Test dengan 56 fitur dan 100 trees:0.91
Recall model RFE data Train dengan 56 fitur dan 100 trees:1.0
Recall model RFE data Test dengan 56 fitur dan 100 trees:0.91
 
Total waktu:  1.4110021591186523
Akurasi model RFE data Train dengan 57 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 57 fitur dan 100 trees:0.88
Precision model RFE data Train dengan 57 fit

Akurasi model RFE data Train dengan 73 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 73 fitur dan 100 trees:0.91
Precision model RFE data Train dengan 73 fitur dan 100 trees:1.0
Precision model RFE data Test dengan 73 fitur dan 100 trees:0.91
Recall model RFE data Train dengan 73 fitur dan 100 trees:1.0
Recall model RFE data Test dengan 73 fitur dan 100 trees:0.91
 
Total waktu:  1.4264435768127441
Akurasi model RFE data Train dengan 74 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 74 fitur dan 100 trees:0.91
Precision model RFE data Train dengan 74 fitur dan 100 trees:1.0
Precision model RFE data Test dengan 74 fitur dan 100 trees:0.91
Recall model RFE data Train dengan 74 fitur dan 100 trees:1.0
Recall model RFE data Test dengan 74 fitur dan 100 trees:0.91
 
Total waktu:  1.5143215656280518
Akurasi model RFE data Train dengan 75 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 75 fitur dan 100 trees:0.93
Precision model RFE data Train dengan 75 fit

Akurasi model RFE data Train dengan 91 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 91 fitur dan 100 trees:0.89
Precision model RFE data Train dengan 91 fitur dan 100 trees:1.0
Precision model RFE data Test dengan 91 fitur dan 100 trees:0.89
Recall model RFE data Train dengan 91 fitur dan 100 trees:1.0
Recall model RFE data Test dengan 91 fitur dan 100 trees:0.89
 
Total waktu:  1.4644801616668701
Akurasi model RFE data Train dengan 92 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 92 fitur dan 100 trees:0.86
Precision model RFE data Train dengan 92 fitur dan 100 trees:1.0
Precision model RFE data Test dengan 92 fitur dan 100 trees:0.86
Recall model RFE data Train dengan 92 fitur dan 100 trees:1.0
Recall model RFE data Test dengan 92 fitur dan 100 trees:0.86
 
Total waktu:  1.4341332912445068
Akurasi model RFE data Train dengan 93 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 93 fitur dan 100 trees:0.88
Precision model RFE data Train dengan 93 fit

In [28]:
%%time

#tentukan metode scoring yang digunakan
scoring_rfe = {'acc': 'accuracy',
               'prec_micro': 'precision_micro',
               'rec_micro': 'recall_micro'}

#tentukan total fitur dan trees yang digunakan dalam proses klasifikasi ini
n_feat = range(1, 101)
n_trees = [150]

max_acc = 0
max_prec = 0
max_rec = 0
for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        start_time = time.time()
        
        #ambil n fitur input hasil seleksi fitur MI
        x_train_selected = x_train[fisher_ranking.nlargest(100,columns="Fisher_Ranking").iloc[0:nfeat, 0]]
        x_test_selected = x_test[fisher_ranking.nlargest(100,columns="Fisher_Ranking").iloc[0:nfeat, 0]]

        #Create a Gaussian Classifier
        clf_rfe = RandomForestClassifier(n_estimators=ntrees)
        
        #Train the model using the training sets
        clf_rfe.fit(x_train_selected, y_train)
        y_pred_rfe=clf_rfe.predict(x_test_selected)
        
        #hitung score model dari data train
        scores_rfe = cross_validate(clf_rfe, x_train_selected, y_train, scoring=scoring_rfe, cv=cv, return_train_score=True)
        
        print("Akurasi model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_acc'].mean(), 2)))
        print("Akurasi model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.accuracy_score(y_test, y_pred_rfe), 2)))
        print("Precision model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_prec_micro'].mean(), 2)))
        print("Precision model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2)))
        print("Recall model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_rec_micro'].mean(), 2)))
        print("Recall model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2)))
        print(" ")
        
        
        
        if(round(metrics.accuracy_score(y_test, y_pred_rfe), 2) > max_acc):
            max_acc = round(metrics.accuracy_score(y_test, y_pred_rfe), 2)
        
        if(round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2) > max_prec):
            max_prec = round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2)
            
        if(round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2) > max_rec):
            max_rec = round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2)
            
        end_time = time.time()
        print("Total waktu: ", end_time - start_time)

print("====Nilai Performa Tertinggi====")
print("Nilai akurasi model tertinggi: " + str(max_acc))
print("Nilai presisi model tertinggi: " + str(max_prec))
print("Nilai Recall model tertinggi: " + str(max_rec))

Akurasi model RFE data Train dengan 1 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 1 fitur dan 150 trees:0.84
Precision model RFE data Train dengan 1 fitur dan 150 trees:1.0
Precision model RFE data Test dengan 1 fitur dan 150 trees:0.84
Recall model RFE data Train dengan 1 fitur dan 150 trees:1.0
Recall model RFE data Test dengan 1 fitur dan 150 trees:0.84
 
Total waktu:  1.9331796169281006
Akurasi model RFE data Train dengan 2 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 2 fitur dan 150 trees:0.82
Precision model RFE data Train dengan 2 fitur dan 150 trees:1.0
Precision model RFE data Test dengan 2 fitur dan 150 trees:0.82
Recall model RFE data Train dengan 2 fitur dan 150 trees:1.0
Recall model RFE data Test dengan 2 fitur dan 150 trees:0.82
 
Total waktu:  1.832563877105713
Akurasi model RFE data Train dengan 3 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 3 fitur dan 150 trees:0.82
Precision model RFE data Train dengan 3 fitur dan 150 trees

Akurasi model RFE data Train dengan 19 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 19 fitur dan 150 trees:0.84
Precision model RFE data Train dengan 19 fitur dan 150 trees:1.0
Precision model RFE data Test dengan 19 fitur dan 150 trees:0.84
Recall model RFE data Train dengan 19 fitur dan 150 trees:1.0
Recall model RFE data Test dengan 19 fitur dan 150 trees:0.84
 
Total waktu:  1.9927968978881836
Akurasi model RFE data Train dengan 20 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 20 fitur dan 150 trees:0.84
Precision model RFE data Train dengan 20 fitur dan 150 trees:1.0
Precision model RFE data Test dengan 20 fitur dan 150 trees:0.84
Recall model RFE data Train dengan 20 fitur dan 150 trees:1.0
Recall model RFE data Test dengan 20 fitur dan 150 trees:0.84
 
Total waktu:  1.953709363937378
Akurasi model RFE data Train dengan 21 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 21 fitur dan 150 trees:0.88
Precision model RFE data Train dengan 21 fitu

Akurasi model RFE data Train dengan 37 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 37 fitur dan 150 trees:0.89
Precision model RFE data Train dengan 37 fitur dan 150 trees:1.0
Precision model RFE data Test dengan 37 fitur dan 150 trees:0.89
Recall model RFE data Train dengan 37 fitur dan 150 trees:1.0
Recall model RFE data Test dengan 37 fitur dan 150 trees:0.89
 
Total waktu:  2.0397558212280273
Akurasi model RFE data Train dengan 38 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 38 fitur dan 150 trees:0.91
Precision model RFE data Train dengan 38 fitur dan 150 trees:1.0
Precision model RFE data Test dengan 38 fitur dan 150 trees:0.91
Recall model RFE data Train dengan 38 fitur dan 150 trees:1.0
Recall model RFE data Test dengan 38 fitur dan 150 trees:0.91
 
Total waktu:  2.0264105796813965
Akurasi model RFE data Train dengan 39 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 39 fitur dan 150 trees:0.89
Precision model RFE data Train dengan 39 fit

Akurasi model RFE data Train dengan 55 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 55 fitur dan 150 trees:0.88
Precision model RFE data Train dengan 55 fitur dan 150 trees:1.0
Precision model RFE data Test dengan 55 fitur dan 150 trees:0.88
Recall model RFE data Train dengan 55 fitur dan 150 trees:1.0
Recall model RFE data Test dengan 55 fitur dan 150 trees:0.88
 
Total waktu:  2.0534651279449463
Akurasi model RFE data Train dengan 56 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 56 fitur dan 150 trees:0.91
Precision model RFE data Train dengan 56 fitur dan 150 trees:1.0
Precision model RFE data Test dengan 56 fitur dan 150 trees:0.91
Recall model RFE data Train dengan 56 fitur dan 150 trees:1.0
Recall model RFE data Test dengan 56 fitur dan 150 trees:0.91
 
Total waktu:  2.1387600898742676
Akurasi model RFE data Train dengan 57 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 57 fitur dan 150 trees:0.89
Precision model RFE data Train dengan 57 fit

Akurasi model RFE data Train dengan 73 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 73 fitur dan 150 trees:0.91
Precision model RFE data Train dengan 73 fitur dan 150 trees:1.0
Precision model RFE data Test dengan 73 fitur dan 150 trees:0.91
Recall model RFE data Train dengan 73 fitur dan 150 trees:1.0
Recall model RFE data Test dengan 73 fitur dan 150 trees:0.91
 
Total waktu:  2.1830508708953857
Akurasi model RFE data Train dengan 74 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 74 fitur dan 150 trees:0.91
Precision model RFE data Train dengan 74 fitur dan 150 trees:1.0
Precision model RFE data Test dengan 74 fitur dan 150 trees:0.91
Recall model RFE data Train dengan 74 fitur dan 150 trees:1.0
Recall model RFE data Test dengan 74 fitur dan 150 trees:0.91
 
Total waktu:  2.093780517578125
Akurasi model RFE data Train dengan 75 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 75 fitur dan 150 trees:0.91
Precision model RFE data Train dengan 75 fitu

Akurasi model RFE data Train dengan 91 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 91 fitur dan 150 trees:0.91
Precision model RFE data Train dengan 91 fitur dan 150 trees:1.0
Precision model RFE data Test dengan 91 fitur dan 150 trees:0.91
Recall model RFE data Train dengan 91 fitur dan 150 trees:1.0
Recall model RFE data Test dengan 91 fitur dan 150 trees:0.91
 
Total waktu:  2.1513869762420654
Akurasi model RFE data Train dengan 92 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 92 fitur dan 150 trees:0.88
Precision model RFE data Train dengan 92 fitur dan 150 trees:1.0
Precision model RFE data Test dengan 92 fitur dan 150 trees:0.88
Recall model RFE data Train dengan 92 fitur dan 150 trees:1.0
Recall model RFE data Test dengan 92 fitur dan 150 trees:0.88
 
Total waktu:  2.0885422229766846
Akurasi model RFE data Train dengan 93 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 93 fitur dan 150 trees:0.88
Precision model RFE data Train dengan 93 fit

In [31]:
%%time

#tentukan metode scoring yang digunakan
scoring_rfe = {'acc': 'accuracy',
               'prec_micro': 'precision_micro',
               'rec_micro': 'recall_micro'}

#tentukan total fitur dan trees yang digunakan dalam proses klasifikasi ini
n_feat = range(1, 101)
n_trees = [200]

max_acc = 0
max_prec = 0
max_rec = 0

for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        start_time = time.time()
        
        #ambil n fitur input hasil seleksi fitur MI
        x_train_selected = x_train[fisher_ranking.nlargest(100,columns="Fisher_Ranking").iloc[0:nfeat, 0]]
        x_test_selected = x_test[fisher_ranking.nlargest(100,columns="Fisher_Ranking").iloc[0:nfeat, 0]]

        #Create a Gaussian Classifier
        clf_rfe = RandomForestClassifier(n_estimators=ntrees)
        
        #Train the model using the training sets
        clf_rfe.fit(x_train_selected, y_train)
        y_pred_rfe=clf_rfe.predict(x_test_selected)
        
        #hitung score model dari data train
        scores_rfe = cross_validate(clf_rfe, x_train_selected, y_train, scoring=scoring_rfe, cv=cv, return_train_score=True)
        
        print("Akurasi model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_acc'].mean(), 2)))
        print("Akurasi model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.accuracy_score(y_test, y_pred_rfe), 2)))
        print("Precision model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_prec_micro'].mean(), 2)))
        print("Precision model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2)))
        print("Recall model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_rec_micro'].mean(), 2)))
        print("Recall model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2)))
        print(" ")
        
        
        if(round(metrics.accuracy_score(y_test, y_pred_rfe), 2) > max_acc):
            max_acc = round(metrics.accuracy_score(y_test, y_pred_rfe), 2)
        
        if(round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2) > max_prec):
            max_prec = round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2)
            
        if(round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2) > max_rec):
            max_rec = round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2)
            
        end_time = time.time()
        print("Total waktu: ", end_time - start_time)

print("====Nilai Performa Tertinggi====")
print("Nilai akurasi model tertinggi: " + str(max_acc))
print("Nilai presisi model tertinggi: " + str(max_prec))
print("Nilai Recall model tertinggi: " + str(max_rec))

Akurasi model RFE data Train dengan 1 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 1 fitur dan 200 trees:0.84
Precision model RFE data Train dengan 1 fitur dan 200 trees:1.0
Precision model RFE data Test dengan 1 fitur dan 200 trees:0.84
Recall model RFE data Train dengan 1 fitur dan 200 trees:1.0
Recall model RFE data Test dengan 1 fitur dan 200 trees:0.84
 
Total waktu:  2.5829789638519287
Akurasi model RFE data Train dengan 2 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 2 fitur dan 200 trees:0.82
Precision model RFE data Train dengan 2 fitur dan 200 trees:1.0
Precision model RFE data Test dengan 2 fitur dan 200 trees:0.82
Recall model RFE data Train dengan 2 fitur dan 200 trees:1.0
Recall model RFE data Test dengan 2 fitur dan 200 trees:0.82
 
Total waktu:  2.4601802825927734
Akurasi model RFE data Train dengan 3 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 3 fitur dan 200 trees:0.82
Precision model RFE data Train dengan 3 fitur dan 200 tree

Akurasi model RFE data Train dengan 19 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 19 fitur dan 200 trees:0.84
Precision model RFE data Train dengan 19 fitur dan 200 trees:1.0
Precision model RFE data Test dengan 19 fitur dan 200 trees:0.84
Recall model RFE data Train dengan 19 fitur dan 200 trees:1.0
Recall model RFE data Test dengan 19 fitur dan 200 trees:0.84
 
Total waktu:  2.608567953109741
Akurasi model RFE data Train dengan 20 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 20 fitur dan 200 trees:0.84
Precision model RFE data Train dengan 20 fitur dan 200 trees:1.0
Precision model RFE data Test dengan 20 fitur dan 200 trees:0.84
Recall model RFE data Train dengan 20 fitur dan 200 trees:1.0
Recall model RFE data Test dengan 20 fitur dan 200 trees:0.84
 
Total waktu:  2.587817668914795
Akurasi model RFE data Train dengan 21 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 21 fitur dan 200 trees:0.84
Precision model RFE data Train dengan 21 fitur

Akurasi model RFE data Train dengan 37 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 37 fitur dan 200 trees:0.89
Precision model RFE data Train dengan 37 fitur dan 200 trees:1.0
Precision model RFE data Test dengan 37 fitur dan 200 trees:0.89
Recall model RFE data Train dengan 37 fitur dan 200 trees:1.0
Recall model RFE data Test dengan 37 fitur dan 200 trees:0.89
 
Total waktu:  2.6918578147888184
Akurasi model RFE data Train dengan 38 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 38 fitur dan 200 trees:0.89
Precision model RFE data Train dengan 38 fitur dan 200 trees:1.0
Precision model RFE data Test dengan 38 fitur dan 200 trees:0.89
Recall model RFE data Train dengan 38 fitur dan 200 trees:1.0
Recall model RFE data Test dengan 38 fitur dan 200 trees:0.89
 
Total waktu:  2.693493127822876
Akurasi model RFE data Train dengan 39 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 39 fitur dan 200 trees:0.89
Precision model RFE data Train dengan 39 fitu

Akurasi model RFE data Train dengan 55 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 55 fitur dan 200 trees:0.93
Precision model RFE data Train dengan 55 fitur dan 200 trees:1.0
Precision model RFE data Test dengan 55 fitur dan 200 trees:0.93
Recall model RFE data Train dengan 55 fitur dan 200 trees:1.0
Recall model RFE data Test dengan 55 fitur dan 200 trees:0.93
 
Total waktu:  2.7479851245880127
Akurasi model RFE data Train dengan 56 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 56 fitur dan 200 trees:0.91
Precision model RFE data Train dengan 56 fitur dan 200 trees:1.0
Precision model RFE data Test dengan 56 fitur dan 200 trees:0.91
Recall model RFE data Train dengan 56 fitur dan 200 trees:1.0
Recall model RFE data Test dengan 56 fitur dan 200 trees:0.91
 
Total waktu:  2.7753689289093018
Akurasi model RFE data Train dengan 57 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 57 fitur dan 200 trees:0.89
Precision model RFE data Train dengan 57 fit

Akurasi model RFE data Train dengan 73 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 73 fitur dan 200 trees:0.91
Precision model RFE data Train dengan 73 fitur dan 200 trees:1.0
Precision model RFE data Test dengan 73 fitur dan 200 trees:0.91
Recall model RFE data Train dengan 73 fitur dan 200 trees:1.0
Recall model RFE data Test dengan 73 fitur dan 200 trees:0.91
 
Total waktu:  2.7748003005981445
Akurasi model RFE data Train dengan 74 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 74 fitur dan 200 trees:0.91
Precision model RFE data Train dengan 74 fitur dan 200 trees:1.0
Precision model RFE data Test dengan 74 fitur dan 200 trees:0.91
Recall model RFE data Train dengan 74 fitur dan 200 trees:1.0
Recall model RFE data Test dengan 74 fitur dan 200 trees:0.91
 
Total waktu:  2.7509829998016357
Akurasi model RFE data Train dengan 75 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 75 fitur dan 200 trees:0.91
Precision model RFE data Train dengan 75 fit

Akurasi model RFE data Train dengan 91 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 91 fitur dan 200 trees:0.88
Precision model RFE data Train dengan 91 fitur dan 200 trees:1.0
Precision model RFE data Test dengan 91 fitur dan 200 trees:0.88
Recall model RFE data Train dengan 91 fitur dan 200 trees:1.0
Recall model RFE data Test dengan 91 fitur dan 200 trees:0.88
 
Total waktu:  2.863689422607422
Akurasi model RFE data Train dengan 92 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 92 fitur dan 200 trees:0.88
Precision model RFE data Train dengan 92 fitur dan 200 trees:1.0
Precision model RFE data Test dengan 92 fitur dan 200 trees:0.88
Recall model RFE data Train dengan 92 fitur dan 200 trees:1.0
Recall model RFE data Test dengan 92 fitur dan 200 trees:0.88
 
Total waktu:  2.8486664295196533
Akurasi model RFE data Train dengan 93 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 93 fitur dan 200 trees:0.88
Precision model RFE data Train dengan 93 fitu

# Klasifikasi Support Vector Machine

Untuk train dan test set, serta cv-nya ambil dari yang Random Forest Classifier, sama aja 

## Testing Performa Model Support Vector Machine

In [22]:
%%time

#tentukan metode scoring yang digunakan
scoring_svm = {'acc': 'accuracy',
               'prec_micro': 'precision_micro',
               'rec_micro': 'recall_micro'}

#tentukan total fitur yang digunakan dalam proses klasifikasi SVM ini
n_feat = range(1, 101)

for nfeat in n_feat:
        print("==================================================")
        start_time = time.time()
        
        #ambil n fitur input hasil seleksi fitur MI
        x_train_selected = x_train[fisher_ranking.nlargest(100,columns="Fisher_Ranking").iloc[0:nfeat, 0]]
        x_test_selected = x_test[fisher_ranking.nlargest(100,columns="Fisher_Ranking").iloc[0:nfeat, 0]]


        #Create a Support Vector Classifier
        clf_svm = svm.SVC()

        #Train the model using the training sets
        clf_svm.fit(x_train_selected, y_train)
        y_pred_svm = clf_svm.predict(x_test_selected)

        #hitung score model dari data train
        scores_svm = cross_validate(clf_svm, x_train_selected, y_train, scoring=scoring_svm, cv=cv, return_train_score=True)

        print("akurasi model SVM data Train dengan " + str(nfeat) + " fitur: " 
              + str(round(scores_svm['train_acc'].mean(), 2)))
        print("akurasi model SVM data Test dengan " + str(nfeat) + " fitur: " 
              + str(round(metrics.accuracy_score(y_test, y_pred_svm), 2)))
        print("Precision model SVM data Train dengan " + str(nfeat) + " fitur:"
              + str(round(scores_svm['train_prec_micro'].mean(), 2)))
        print("Precision model SVM data Test dengan " + str(nfeat) + " fitur:"
              + str(round(metrics.precision_score(y_test, y_pred_svm, average='micro'), 2)))
        print("Recall model SVM data Train dengan " + str(nfeat) + " fitur:"
              + str(round(scores_svm['train_rec_micro'].mean(), 2)))
        print("Recall model SVM data Test dengan " + str(nfeat) + " fitur:"
              + str(round(metrics.recall_score(y_test, y_pred_svm, average='micro'), 2)))
        print(" ")

        
        max_acc = 0
        max_prec = 0
        max_rec = 0
        
        if(round(metrics.accuracy_score(y_test, y_pred_svm), 2) > max_acc):
            max_acc = round(metrics.accuracy_score(y_test, y_pred_svm), 2)
        
        if(round(metrics.precision_score(y_test, y_pred_svm, average='micro'), 2) > max_prec):
            max_prec = round(metrics.precision_score(y_test, y_pred_svm, average='micro'), 2)
            
        if(round(metrics.recall_score(y_test, y_pred_svm, average='micro'), 2) > max_rec):
            max_rec = round(metrics.recall_score(y_test, y_pred_svm, average='micro'), 2)
            
        end_time = time.time()
        print("Total waktu: ", end_time - start_time)

print("====Nilai Performa Tertinggi====")
print("Nilai akurasi model tertinggi: " + str(max_acc))
print("Nilai presisi model tertinggi: " + str(max_prec))
print("Nilai Recall model tertinggi: " + str(max_rec))

akurasi model SVM data Train dengan 1 fitur: 0.72
akurasi model SVM data Test dengan 1 fitur: 0.64
Precision model SVM data Train dengan 1 fitur:0.72
Precision model SVM data Test dengan 1 fitur:0.64
Recall model SVM data Train dengan 1 fitur:0.72
Recall model SVM data Test dengan 1 fitur:0.64
 
Total waktu:  0.14692354202270508
akurasi model SVM data Train dengan 2 fitur: 0.71
akurasi model SVM data Test dengan 2 fitur: 0.64
Precision model SVM data Train dengan 2 fitur:0.71
Precision model SVM data Test dengan 2 fitur:0.64
Recall model SVM data Train dengan 2 fitur:0.71
Recall model SVM data Test dengan 2 fitur:0.64
 
Total waktu:  0.09450936317443848
akurasi model SVM data Train dengan 3 fitur: 0.71
akurasi model SVM data Test dengan 3 fitur: 0.64
Precision model SVM data Train dengan 3 fitur:0.71
Precision model SVM data Test dengan 3 fitur:0.64
Recall model SVM data Train dengan 3 fitur:0.71
Recall model SVM data Test dengan 3 fitur:0.64
 
Total waktu:  0.08847570419311523
akurasi

akurasi model SVM data Train dengan 25 fitur: 0.71
akurasi model SVM data Test dengan 25 fitur: 0.64
Precision model SVM data Train dengan 25 fitur:0.71
Precision model SVM data Test dengan 25 fitur:0.64
Recall model SVM data Train dengan 25 fitur:0.71
Recall model SVM data Test dengan 25 fitur:0.64
 
Total waktu:  0.11568689346313477
akurasi model SVM data Train dengan 26 fitur: 0.71
akurasi model SVM data Test dengan 26 fitur: 0.64
Precision model SVM data Train dengan 26 fitur:0.71
Precision model SVM data Test dengan 26 fitur:0.64
Recall model SVM data Train dengan 26 fitur:0.71
Recall model SVM data Test dengan 26 fitur:0.64
 
Total waktu:  0.09976053237915039
akurasi model SVM data Train dengan 27 fitur: 0.71
akurasi model SVM data Test dengan 27 fitur: 0.64
Precision model SVM data Train dengan 27 fitur:0.71
Precision model SVM data Test dengan 27 fitur:0.64
Recall model SVM data Train dengan 27 fitur:0.71
Recall model SVM data Test dengan 27 fitur:0.64
 
Total waktu:  0.0953147

akurasi model SVM data Train dengan 48 fitur: 0.71
akurasi model SVM data Test dengan 48 fitur: 0.66
Precision model SVM data Train dengan 48 fitur:0.71
Precision model SVM data Test dengan 48 fitur:0.66
Recall model SVM data Train dengan 48 fitur:0.71
Recall model SVM data Test dengan 48 fitur:0.66
 
Total waktu:  0.11206173896789551
akurasi model SVM data Train dengan 49 fitur: 0.71
akurasi model SVM data Test dengan 49 fitur: 0.66
Precision model SVM data Train dengan 49 fitur:0.71
Precision model SVM data Test dengan 49 fitur:0.66
Recall model SVM data Train dengan 49 fitur:0.71
Recall model SVM data Test dengan 49 fitur:0.66
 
Total waktu:  0.10736584663391113
akurasi model SVM data Train dengan 50 fitur: 0.71
akurasi model SVM data Test dengan 50 fitur: 0.66
Precision model SVM data Train dengan 50 fitur:0.71
Precision model SVM data Test dengan 50 fitur:0.66
Recall model SVM data Train dengan 50 fitur:0.71
Recall model SVM data Test dengan 50 fitur:0.66
 
Total waktu:  0.1036927

akurasi model SVM data Train dengan 70 fitur: 0.71
akurasi model SVM data Test dengan 70 fitur: 0.66
Precision model SVM data Train dengan 70 fitur:0.71
Precision model SVM data Test dengan 70 fitur:0.66
Recall model SVM data Train dengan 70 fitur:0.71
Recall model SVM data Test dengan 70 fitur:0.66
 
Total waktu:  0.11362171173095703
akurasi model SVM data Train dengan 71 fitur: 0.71
akurasi model SVM data Test dengan 71 fitur: 0.66
Precision model SVM data Train dengan 71 fitur:0.71
Precision model SVM data Test dengan 71 fitur:0.66
Recall model SVM data Train dengan 71 fitur:0.71
Recall model SVM data Test dengan 71 fitur:0.66
 
Total waktu:  0.11064267158508301
akurasi model SVM data Train dengan 72 fitur: 0.71
akurasi model SVM data Test dengan 72 fitur: 0.66
Precision model SVM data Train dengan 72 fitur:0.71
Precision model SVM data Test dengan 72 fitur:0.66
Recall model SVM data Train dengan 72 fitur:0.71
Recall model SVM data Test dengan 72 fitur:0.66
 
Total waktu:  0.1343135

akurasi model SVM data Train dengan 92 fitur: 0.71
akurasi model SVM data Test dengan 92 fitur: 0.66
Precision model SVM data Train dengan 92 fitur:0.71
Precision model SVM data Test dengan 92 fitur:0.66
Recall model SVM data Train dengan 92 fitur:0.71
Recall model SVM data Test dengan 92 fitur:0.66
 
Total waktu:  0.12310600280761719
akurasi model SVM data Train dengan 93 fitur: 0.71
akurasi model SVM data Test dengan 93 fitur: 0.66
Precision model SVM data Train dengan 93 fitur:0.71
Precision model SVM data Test dengan 93 fitur:0.66
Recall model SVM data Train dengan 93 fitur:0.71
Recall model SVM data Test dengan 93 fitur:0.66
 
Total waktu:  0.11908078193664551
akurasi model SVM data Train dengan 94 fitur: 0.71
akurasi model SVM data Test dengan 94 fitur: 0.66
Precision model SVM data Train dengan 94 fitur:0.71
Precision model SVM data Test dengan 94 fitur:0.66
Recall model SVM data Train dengan 94 fitur:0.71
Recall model SVM data Test dengan 94 fitur:0.66
 
Total waktu:  0.1387605

# Misc

In [17]:
#ini descending
#bikin dataframe buat nyimpan fitur beserta rankingnya
df_fisher = pd.DataFrame(x_train.columns, columns =['Fitur'])
df_fisher.insert(1, "Fisher Ranking", fisher_score, True)

df_fisher.sort_values(by=['Fisher Ranking'])

,Fitur,Fisher Ranking
1556,2500.2,0
1555,2497.8,1
1554,2495.4,2
1553,2493,3
1552,2490.6,4
...,...,...
319,1140.2,1552
323,1142.2,1553
325,1143.2,1554
326,1143.7,1555
